<a href="https://colab.research.google.com/github/meganificent/Language-Style-Transfer/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.	Insert English phrase
a.	Tokenize English phrase --> get POS array of tuple
2.	Translate English phrase to Spanish using Google Translate
3.	Token Spanish phrase
4.	Spaghetti tokenized Spanish phrase --> output array of tuples with Spanish word as the key and pos as the value
5.	Go through each key of POS English phrase and compare to keys of POS Spanish phrases --> move all English object intervals so that they are in the same positions as their translated Spanish word counterparts
6.	Merge rearranged array of objects so that their keys produce one potentially grammatically incoherent English sentence

Special cases: "a" in spanish, "to" for location in English --> brainstorm placeholders for these kinds of things

In [42]:
spanglishMachine("I have not had any index errors with my matrix bullshit yet")

before: I have not had any index errors with my matrix bullshit yet
after: have not had any errors of index with my bullshit of matrix yet


### Stuff to Run

In [1]:
import matplotlib
matplotlib.use('Agg')
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import numpy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
from __future__ import print_function

In [4]:
import spaghetti as sp
import nltk
nltk.download('cess_esp')
mytagger = sp.CESSTagger()
mytagger_uni = mytagger.uni
mytagger_bi = mytagger.bi

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.


*** First-time use of cess tagger ***
Training tagger ...
Tagger trained with cess using UnigramTagger and BigramTagger.
Tagger trained with cess_nomwe using UnigramTagger and BigramTagger.


In [5]:
from nltk.corpus import wordnet
syns = wordnet.synsets("up")
print(syns[2].lemmas()[0].antonyms())

[]


In [6]:
	synonyms = []
	antonyms = []

	for syn in wordnet.synsets("none"):
		for l in syn.lemmas():
			synonyms.append(l.name())
			if l.antonyms():
				 antonyms.append(l.antonyms()[0].name())

	print(set(synonyms))
	print(set(antonyms))

{'none'}
set()


In [7]:
!pip install googletrans==3.1.0a0
import googletrans
print(googletrans.LANGUAGES)
from googletrans import Translator

     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 6.5MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=c0be3eb2beb9b5e138e5c3b961b59bdfec73c1fa08a3bab459aa945e76d171a4
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans
{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esp

In [8]:
translator = Translator(service_urls=['translate.googleapis.com'])


### **Google Translate**

In [9]:
result = translator.translate("el perro blanco corrió", dest="en")
print(result.text)
print(result.src)
print(result.dest)


the white dog ran
es
en


### Spanglish Machine

In [28]:
spanglishMachine("white coffee cup spilled him")

before: white coffee cup spilled him
after: him spilled the cup of coffee white


In [29]:
spanglishMachine("Megan wrote it")

before: Megan wrote it
after: it wrote Megan


In [30]:
spanglishMachine("I write but the hot sun stays down")

before: I write but the hot sun stays down
after: write but stays the sun hot down


In [33]:
spanglishMachine("I wrote while the blue wolf cried to the moon")

before: I wrote while the blue wolf cried to the moon
after: wrote while cried the wolf blue to the moon


In [34]:
spanglishMachine("I wrote while blue coffee cups cried to the moon")

before: I wrote while blue coffee cups cried to the moon
after: wrote while cried the cups of coffee blue to the moon


In [35]:
spanglishMachine("blue sister's heart cried")

before: blue sister 's heart cried
after: the heart cried of the sister blue


In [36]:
spanglishMachine("I wrote while Joanna's blue coffee cups cried to it")

before: I wrote while Joanna 's blue coffee cups cried to it
after: wrote while cried the cups of coffee blue of Joanna to it


In [37]:
spanglishMachine("she wrote it to him")

before: she wrote it to him
after: she him it wrote


In [18]:
def spanglishMachine(string): 
  origTokenized = nltk.word_tokenize(string)

  #determine language of inserted string and feed into correct translation function
  
  translated = translator.translate(string, dest = "en") #initially translate to english
  if translated.src == "en": #if the original language was english
    tagged = nltk.pos_tag(origTokenized) #tag POS using nltk
    #print("Original:", tagged)
    
    #old sentence for testing
    originalString = " ".join([word[0] for word in tagged])
    #print("before:", originalString)

    translatedArray = toSpanishStyle(tagged) #transfer to Spanish
    
    #new sentence for testing
    translatedString = " ".join([word[0] for word in translatedArray])
    print("before:", originalString)
    print("after:", translatedString)

  else: #if original language was spanish
    tagged = sp.pos_tag(origTokenized) #tag POS using spaghetti
    print("Original:", tagged)
    translated = toEnglishStyle(tagged) #transfer to English
    #print(translated)



In [ ]:
fi = ["hi", "my", "tired", "mind"]
fi[3:3] = "beautiful", "lost"
print(fi)
fi[4:4] = fi[1:3]
print(fi)
del fi[1:3]
print(fi)
print(len(fi))

['hi', 'my', 'tired', 'beautiful', 'lost', 'mind']
['hi', 'my', 'tired', 'beautiful', 'my', 'tired', 'lost', 'mind']
['hi', 'beautiful', 'my', 'tired', 'lost', 'mind']
6


In [16]:
def toSpanishStyle(taggedWords):
  #Rule 1: Remove P
  #taggedWords = [word for word in taggedWords if word[0] != "I"]

  #Eventually need to figure out how to treat posessive proper nouns --> should be paired with noun. Need to conver before Attributive Nouns step

  #Rule 4: Possessive Nouns After What they Possess
  idx = 0
  while idx < len(taggedWords)-1:
    if taggedWords[idx][1] == "POS": #if current word is a possessive ending ("'s")
      #print(idx)
      #possessive + noun and all its qualifiers
      posNounEnd = idx+1
      #print("posNounEnd = ", taggedWords[posNounEnd])
      posNounStart = getNounStart(posNounEnd, taggedWords)
      #noun + all its qualifiers of the possessed noun
      ownedNounStart = idx+1
      #print("ownedNounStart = ", taggedWords[ownedNounStart])
      ownedNounEnd = idx+2 #added 1 for the sake of indexing later
      if idx+2 < len(taggedWords): #avoid runtime errors
        if taggedWords[idx+2][1] == "NN":
          ownedNounEnd = idx+3
          if idx+3 < len(taggedWords): #avoid runtime errors
            if taggedWords[idx+3][1] == "NN":
              ownedNounEnd = idx+4
      #move posNoun after ownedNoun
      taggedWords[ownedNounEnd+1:ownedNounEnd+1] = nltk.pos_tag(nltk.word_tokenize("of")) #insert "of" after posessed noun
      taggedWords[ownedNounEnd+2:ownedNounEnd+2] = taggedWords[posNounStart:posNounEnd-1] #add posessive noun stuff (minus 's) after owned noun stuff's "of"
      del taggedWords[posNounStart:posNounEnd] #remove original pos noun stuff

      idx+=1 #so dont recount things
    
    idx +=1
  #print("Posessive Nouns pt 1:", taggedWords)

  #Rule 3.5: Noun after Verb When It's a Verb applied to the second noun in the Sentence (because switch more common with long sentences)
  idx = 0
  numSubjects = 0
  while idx < len(taggedWords)-1:
    if (taggedWords[idx][1][:2] == "NN") | (taggedWords[idx][1] == "PRP"): #if current word is a noun
      if taggedWords[idx+1][1][:2] == "VB":        
        numSubjects += 1
        if numSubjects == 2: #if currently on second noun with verb
          nounEnd = idx+1 # idx-1 + 1 for the sake of indexing later
          nounStart = getNounStart(nounEnd, taggedWords) #where noun's qualifier's start
          
         # if nounEnd + 2 <len(taggedWords):
         #   if taggedWords[nounEnd+1][0] == "to": #if now has "to" after it
         #     if taggedWords[nounEnd+2][1] == "PRP": #and 'to' has a pronoun after it
         #       del taggedWords[nounEnd+1]
         #       taggedWords[nounEnd+1] = (taggedWords[nounEnd+1][0], "unmutable")
         #       taggedWords[nounEnd], taggedWords[nounEnd+1] = taggedWords[nounEnd+1], taggedWords[nounEnd] #swap VB and PRP
          taggedWords[idx+2:idx+2] = taggedWords[nounStart:nounEnd] #add noun stuff after verb
          del taggedWords[nounStart:nounEnd] #remove ealier noun elements
    
    idx += 1
  #print("Long Sentence Verb+Noun:", taggedWords)


  #Rule ?: verb+ pronoun1 + to + pronoun2 --> pronoun 2 + pronoun 1 + verb
  idx = 0
  while idx < len(taggedWords) - 3:
    if taggedWords[idx][1][:2] == "VB":
      if taggedWords[idx+1][1] == "PRP":
        if taggedWords[idx+2][0] == "to":
          if taggedWords[idx+3][1] == "PRP":
            taggedWords[idx+1], taggedWords[idx+3] = taggedWords[idx+3], taggedWords[idx+1]
            del taggedWords[idx+2] #delete "to"
            taggedWords[idx:idx] = taggedWords[idx+1:idx+3]
            del taggedWords[idx+3:idx+5]
            idx+=1
    idx+=1
  #print("Remove 'To' and Swap PRP:", taggedWords)

  #Rule 1: Remove Pronouns Directly Before Conjugated Verbs If not 2 or 3 pronoun group before verb
  idx = 0
  while idx < len(taggedWords) - 1:
    if taggedWords[idx][1] == "PRP":
      if taggedWords[idx+1][1][:2] == "VB":
        if idx-1 >= 0:
          if taggedWords[idx-1][1] != "PRP":
            del taggedWords[idx]
        else:
          del taggedWords[idx]
    idx += 1
  #print("Remove Pronouns Before Verbs:", taggedWords)

  #Rule 3: Pronoun + Verb + Noun
  idx = 0
  while idx < len(taggedWords) - 1:
    if taggedWords[idx][1][:2] == "VB": #if current word is a verb
      #print(idx)
      if taggedWords[idx+1][1] == "PRP": #if there's a pronoun directly after the verb
        if idx-1 >= 0: #to avoid runtime errors
          if taggedWords[idx-1][1][:2] == "NN": #if there's a noun right before the verb
            nounEnd = idx # idx-1 + 1 for the sake of indexing later
            nounStart = getNounStart(nounEnd, taggedWords) #where noun's qualifier's start
            if idx+2 < len(taggedWords):
              if taggedWords[idx+2] == "PRP": #if there's a pronoun verb directly after it
                taggedWords[idx-1] = taggedWords[idx+1:idx+3]
                del taggedWords[idx+1:idx+3] #more both pronouns before verb
              else:
                taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx] #swap pronoun and verb positions
            else:
              taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx] #swap pronoun and verb positions
            taggedWords[idx+2:idx+2] = taggedWords[nounStart:nounEnd] #add noun stuff after where pronoun used to be
            del taggedWords[nounStart:nounEnd] #remove ealier noun elements
            idx +=1 #so I don't recount verb

    idx +=1
  #print("Pronoun+Verb+Noun:", taggedWords)



  #Rule 4: Adjectives After Nouns
  idx = 0
  while idx < len(taggedWords) - 1: #for every tuple (word) in Array
    if taggedWords[idx][1][:2] == "JJ": #if current word is an adjective (can tell general tag by first 2 letters of acronym)
      if taggedWords[idx+1][1][:2] == "NN": #if the proceding word is a noun #how do I avoid an error if adjective is last word in list?
        taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx]
    idx += 1
  #print("Adjectives after Nouns:", taggedWords)


  #Rule 5: "The" Before Every Non-Proper Noun
  idx = 0
  while idx < len(taggedWords):
    if (taggedWords[idx][1][:2] == "NN") & (taggedWords[idx][1][:3] != "NNP"): #if word is a noun and not a proper noun
      if idx == 0:
        taggedWords[idx:idx] = nltk.pos_tag(nltk.word_tokenize("the"))
        idx += 1 #so we don't recount nouns
      else: #to avoid runtime errors
        if (taggedWords[idx - 1][1] != "DT") & (taggedWords[idx - 1][1] != "PRP$") & (taggedWords[idx - 1][1][:2] != "NN"): #if noun doesn't already have an "a" or "the" in front of it and if it doesn't have another noun in front of it or a posessive pronouns like "su", waiting to be turned into noun+of+noun
          taggedWords[idx:idx] = nltk.pos_tag(nltk.word_tokenize("the"))
          idx += 1 #so we don't recount nouns
    idx += 1
  #print("Add 'the':", taggedWords)


  #Rule 6: Attributive Nouns
  idx = 0
  while idx < len(taggedWords) - 1:
    if taggedWords[idx][1][:2] == "NN": #if current word is a noun
      if taggedWords[idx+1][1][:2] == "NN": #if directly adjacent word is a noun
        taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx] #swap the nouns
        #print(taggedWords)
        taggedWords[idx+1:idx+1] = nltk.pos_tag(nltk.word_tokenize("of")) #insert "of" between the two nouns
        #print(taggedWords)
    idx += 1
  #print("Attributive Nouns:", taggedWords)

  return(taggedWords)


In [14]:
def getNounStart(endIndex, wordsArray):
  nounStart = endIndex - 1
  while True:
    if endIndex-2 >= 0: #to avoid runtime errors
      if (wordsArray[endIndex-2][1][:2] == "DT") | (wordsArray[endIndex-2][1][:2] == "PRP$") | (wordsArray[endIndex-2][1][:2] == "POS") | (wordsArray[endIndex-2][1][:2] == "JJ") | (wordsArray[endIndex-2][1][:2] == "NN"): #if there's a determiner or adjective next to the noun
        nounStart = endIndex-2
        if endIndex-3 >= 0: #to avoid runtime errors
          if (wordsArray[endIndex-3][1][:2] == "DT") | (wordsArray[endIndex-3][1][:2] == "PRP$") | (wordsArray[endIndex-3][1][:2] == "JJ") | (wordsArray[endIndex-3][1][:2] == "NN"): #if there's a determiner next to the adjective
            nounStart = endIndex-3
            if endIndex-4 >=0:
              if (wordsArray[endIndex-4][1][:2] == "DT") | (wordsArray[endIndex-4][1][:2] == "PRP$") | (wordsArray[endIndex-4][1][:2] == "JJ"):
                nounStart = endIndex-4
                if endIndex-5 >= 0:
                  if (wordsArray[endIndex-5][1][:2] == "DT") | (wordsArray[endIndex-5][1][:2] == "PRP$"):
                    nounStart = endIndex-5
    
    if nounStart-1 >= 0: #to avoid runtime errors
      if wordsArray[nounStart-1][0] == "of":
        endIndex = nounStart-1 #where noun's qualifier's start
      else:
        break
    else:
      break
  return nounStart

In [ ]:
print(nltk.pos_tag(nltk.word_tokenize(("The student's books"))))

[('The', 'DT'), ('student', 'NN'), ("'s", 'POS'), ('books', 'NNS')]


In [ ]:
englishNegAndAff = [("no one", "someone"), ("nothing", "something"), ("none", "some"), ("neither", "also"), ("never", "always"), ("not", "")] #no = exception. If no one --> someone. if no + noun --> none

In [ ]:
spanishNegAndAff = [("nadie", "algien"), ("nada", "algo"), ("ningún", "algún"), ("ninguna", "alguna"), ("ningunas", "algunas"), ("ningunos", "algunos"), ("tampoco", "también"), ("nunca", "siempre")] #exception = no

[nltk Acronym Meanings](https://www.guru99.com/pos-tagging-chunking-nltk.html#:~:text=POS%20Tagging%20(Parts%20of%20Speech,is%20also%20called%20grammatical%20tagging.)

[Grammar Diff Btwn Spanish and English](https://www.thoughtco.com/grammatical-differences-between-spanish-and-english-4119326#:~:text=Word%20order%20is%20less%20fixed,subjunctive%20mood%20than%20English%20does.)

***Sometimes Pronouns Come Before Verbs and Nouns After***

Lo escribió Cervantes.
*   Megan wrote it (the book) --> It (the book) wrote Cervantes
*   NNP VBD PRP --> PRP VBD NNP

"No recuerdo el momento en que salió Pablo"
*   I don't remember the moment in which Pablo left --> I don't remember the moment in which left Pablo
*   (PRP VBP RB VB DT NN IN WDT) NNP VBD --> (same) VBD NNP

***Spanish Uses Double Negatives***

* Apenas come. (She barely eats.)
* Apenas come nada. (She barely eats anything --> she barely eats nothing)
* No tengo ninguno. (I don't have any --> I don't have none)
* Nadie sabe eso. (Nobody knows that.)
* Jamás fumo. (I never smoke.)
* Tampoco comió. (She didn't eat either.)
* Tampoco comió nada. (She didn't eat anything either --> She didn't eat nothing either)
* No habló. (He didn't speak.)
* No dijo nada. (He said nothing --> He didn't say nothing)
* No le dijo nada a nadie. (He didn't say anything to anybody --> He didn't say nothing to nobody)
* No compro ninguno. (I'm not buying any --> I'm not buying none)
* Nunca le compra nada a nadie. (She never buys anything for anybody --> She never buy nothing for nobody)
* No come ni siquiera pan. (He doesn't even eat bread --> He doesnt --honestly I don't even know how to translate this literally)
* Ni siquiera come pan. (He doesn't even eat bread.)

***Can Omit Pronouns if Verb is Conjugated***

* No compro ninguno. (I'm not buying any --> I'm not buying none)
* Nunca le compra nada a nadie. (She never buys anything for anybody --> She never buy nothing for nobody)
* No come ni siquiera pan. (He doesn't even eat bread --> He doesnt --honestly I don't even know how to translate this literally)
* Ni siquiera come pan. (He doesn't even eat bread.)

*Especially* Don't need "I"

***Attributive Nouns***
Not coffee cup --> taza para cafe --> cup of coffee

if two nouns next to each other --> switch nouns and add "of" (para/de) between

* In some cases, this is accomplished by Spanish having adjectival forms that don't exist in English. For example, informático can be the equivalent of "computer" as an adjective, so a computer table is a mesa informática.

***Tacking Pronouns to End of Verbs When "To"***
 * darmelo = give it to me --> give me it
 * escribamostelo = let's write it to you --> let's write you it

In [ ]:
tokenized = nltk.word_tokenize("el perro blanco corrió")
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)

['el', 'perro', 'blanco', 'corrió']
[('el', 'da0ms0'), ('perro', 'ncms000'), ('blanco', 'aq0ms0'), ('corrió', 'vmis3s0')]


In [ ]:
tokenized = nltk.word_tokenize("el perro azul corre")
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)

['el', 'perro', 'azul', 'corre']
[('el', 'da0ms0'), ('perro', 'ncms000'), ('azul', 'aq0cs0'), ('corre', 'vmip3s0')]


In [ ]:
tokenized = nltk.word_tokenize("los perros feo corren")
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)

['los', 'perros', 'feo', 'corren']
[('los', 'da0mp0'), ('perros', 'ncmp000'), ('feo', 'aq0ms0'), ('corren', 'vmip3p0')]


In [ ]:
tokenized = nltk.word_tokenize("una chica bonitas corren")
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)

['una', 'chica', 'bonitas', 'corren']
[('una', 'di0fs0'), ('chica', 'ncfs000'), ('bonitas', None), ('corren', 'vmip3p0')]


In [ ]:
tokenized = nltk.word_tokenize("Argentina corre")
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)

['Argentina', 'corre']
[('Argentina', 'np0000l'), ('corre', 'vmip3s0')]


**Parsing POS Code Meaning**

<u>Nouns</u>

[0:2]
* nc = regular noun
* np = proper noun

[3]
* f = feminine
* m = masculine

[4]
* s = singular
* p = plural

<u>Adjectives</u>

[0:2] = aq

[3]
* f = feminine
* m = masculine
* c = gender nuetral

[4]
* s = singular
* theoretically, p = plural, but spaghetti tagger doesn't seem to have many plural adjectives logged



<u>Determiners</u>

[0] = d

[1]
* a = el, la, los, las
* i = un, una, unos, unas

[2]
* 0 = standard
* 3 = possessive

[3]
* f = feminine
* m = masculine

[4]
* s = singular
* f = feminine

**Notes**
* Spaghetti tagger doesn't know people names
* Its grammar is also wrong surrounding adjectives... doesn't perceive blancos as an adjective (sees it as noun) but percieves blanco as adjective, even when paired with perros
* nor does it keep track of verbs conjugated in first person?


In [106]:
spanglishMachine("se lo corrió")

Original: [('se', 'p0300000'), ('lo', 'da0ns0'), ('corrió', 'vmis3s0')]
Adjectives before Nouns: [('se', 'p0300000'), ('lo', 'da0ns0'), ('corrió', 'vmis3s0')]
Attributive/Possesive Nouns: [('se', 'p0300000'), ('lo', 'da0ns0'), ('corrió', 'vmis3s0')]


In [105]:
spanglishMachine("café de Argentina")

Original: [('café', 'ncms000'), ('de', 'sps00'), ('Argentina', 'np0000l')]
Adjectives before Nouns: [('café', 'ncms000'), ('de', 'sps00'), ('Argentina', 'np0000l')]
1
2
4
Attributive/Possesive Nouns: [('Argentina', 'np0000l'), ("'", 'apostophe'), ('de', 'sps00'), ('café', 'ncms000')]


In [99]:
spanglishMachine("taza de café")

Original: [('taza', 'ncfs000'), ('de', 'sps00'), ('café', 'ncms000')]
Adjectives before Nouns: [('taza', 'ncfs000'), ('de', 'sps00'), ('café', 'ncms000')]
1
2
3
Attributive/Possesive Nouns: [('café', 'ncms000'), ('taza', 'ncfs000')]


In [104]:
def toEnglishStyle(taggedWords):
  #Rule 4: Nouns After Adjectives
  idx = 0
  while idx < len(taggedWords) - 1: #for every tuple (word) in Array
    if taggedWords[idx][1][0] == "n": #if current word is an noun
      if taggedWords[idx+1][1][:2] == "aq": #if the proceding word is a adjective
        taggedWords[idx], taggedWords[idx+1] = taggedWords[idx+1], taggedWords[idx]
    idx += 1
  print("Adjectives before Nouns:", taggedWords)
  #print("under development")

  #omit la from beginning of plural direct objects

  #attributive nouns (2nd noun = improper) and posessive nouns (2nd noun = proper)
  idx = 0
  while idx < len(taggedWords) - 2: #for every tuple (word) in Array
    if taggedWords[idx][1][0] == "n": #if current word is an improper noun
      if taggedWords[idx+1][0] == "de": #if the proceding word is "de"
        if taggedWords[idx+2][1][:2] == "nc": #if the next word is an improper noun -- ATTRIBUTIVE NOUN
          taggedWords[idx], taggedWords[idx+2] = taggedWords[idx+2], taggedWords[idx] #swap 1st and 2nd nouns
          del taggedWords[idx+1] #delete "de"
          #DEAL WITH FEM V MASC DETERMINERS
          #if idx-1 >= 0:
            #if taggedWords[idx-1][1][0][0] == "f":
              #taggedWords[idx-1][1][0][0] == "f":
        elif taggedWords[idx+2][1][:2] == "np": #if 2nd noun = proper --> POSSESSIVE NOUN
          taggedWords[idx], taggedWords[idx+2] = taggedWords[idx+2], taggedWords[idx] #swap nouns
          taggedWords[idx+1:idx+1] = [("'", "apostophe")]
    idx += 1
  print("Attributive/Possesive Nouns:", taggedWords)

  #move around pronouns in front of verbs
  

In [71]:
def getSpanNounEnd(dIndex, wordsArray):
  nounEnd = dIndex+1
  if endIndex+1 < len(wordsArray): #to avoid runtime errors
    if (wordsArray[nounIndex+1][1][0] == "n") | (wordsArray[nounIndex+1][:2][0] == "aq"):
      nounStart = nounEnd+1
      if endIndex-2 >= 0: #to avoid runtime errors
        if (wordsArray[nounIndex+2][1][0] == "n"):
          nounStart = nounEnd+1
  return nounEnd

### **English Data**

In [ ]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

**Syntax** **trees**

In [ ]:
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
### CREATE VIRTUAL DISPLAY ###
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8
Err:1 http://security.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8
  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/x/xorg-server/xvfb_1.19.6-1ubuntu4.8_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
# EXTRA STUFF TO DISPLAY NLTK SYNTAX TREES #  
%matplotlib inline
### INSTALL GHOSTSCRIPT ) ###
!apt install ghostscript python3-tk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.14).
python3-tk is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
chunked_sentence = '(S (NP this tree) (VP (V is) (AdjP pretty)))'

from nltk.tree import Tree
from IPython.display import display
tree = Tree.fromstring(str(chunked_sentence))
display(tree)

TclError: ignored

Tree('S', [Tree('NP', ['this', 'tree']), Tree('VP', [Tree('V', ['is']), Tree('AdjP', ['pretty'])])])

In [ ]:
Sentence = "The white dog ran"
tokenized = nltk.word_tokenize(Sentence)
print(tokenized)
tagged = nltk.pos_tag(tokenized)
print(tagged)
chunkGram = r"""Chunk: {<DT\w?>*<JJ\w?>}""" #creating a chunk
chunkParser = nltk.RegexpParser(chunkGram) #parsing the chunk
#chunked = chunkParser.parse(tagged)
for tree in chunkParser.parse(tagged): #creates a syntax tree for the parsed chunk
    print(tree)
    #tree.draw()



['The', 'white', 'dog', 'ran']
[('The', 'DT'), ('white', 'JJ'), ('dog', 'NN'), ('ran', 'VBD')]
(Chunk The/DT white/JJ)
('dog', 'NN')
('ran', 'VBD')


In [ ]:
Sentence = "white the ran dog."
tokenized = nltk.word_tokenize(Sentence)
print(tokenized)
tagged = nltk.pos_tag(tokenized)
print(tagged)

['white', 'the', 'ran', 'dog', '.']
[('white', 'JJ'), ('the', 'DT'), ('ran', 'NN'), ('dog', 'NN'), ('.', '.')]


### **Spanish Data?**

In [43]:
!git clone https://github.com/alvations/spaghetti-tagger.git

Cloning into 'spaghetti-tagger'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.


In [44]:
from google.colab import drive
drive.mount('/cont
ent/drive')

SyntaxError: ignored

In [45]:
nltk.corpus.cess_esp.words()

['El', 'grupo', 'estatal', 'Electricité_de_France', ...]

In [46]:
nltk.corpus.cess_esp.sents()

[['El', 'grupo', 'estatal', 'Electricité_de_France', '-Fpa-', 'EDF', '-Fpt-', 'anunció', 'hoy', ',', 'jueves', ',', 'la', 'compra', 'del', '51_por_ciento', 'de', 'la', 'empresa', 'mexicana', 'Electricidad_Águila_de_Altamira', '-Fpa-', 'EAA', '-Fpt-', ',', 'creada', 'por', 'el', 'japonés', 'Mitsubishi_Corporation', 'para', 'poner_en_marcha', 'una', 'central', 'de', 'gas', 'de', '495', 'megavatios', '.'], ['Una', 'portavoz', 'de', 'EDF', 'explicó', 'a', 'EFE', 'que', 'el', 'proyecto', 'para', 'la', 'construcción', 'de', 'Altamira_2', ',', 'al', 'norte', 'de', 'Tampico', ',', 'prevé', 'la', 'utilización', 'de', 'gas', 'natural', 'como', 'combustible', 'principal', 'en', 'una', 'central', 'de', 'ciclo', 'combinado', 'que', 'debe', 'empezar', 'a', 'funcionar', 'en', 'mayo_del_2002', '.'], ...]

In [47]:
nltk.download("cess_esp")

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

In [48]:
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt

In [ ]:
# Read the corpus into a list, 
# each entry in the list is one sentence.
cess_sents = cess.tagged_sents()

In [ ]:
# Train the unigram tagger
uni_tag = ut(cess_sents)

In [52]:
spanishSentence = "los perros azules corrion"

In [53]:
#tokenized = spanishSentence.split()
tokenized = nltk.word_tokenize(spanishSentence)
print(tokenized)
tagged1 = sp.pos_tag(tokenized)
print(tagged1)
tagged2 = mytagger_uni.tag(tokenized)
print(tagged2)
tagged4 = mytagger_bi.tag(tokenized)
print(tagged4)
print(tagged4[0])
print(tagged4[0][0]) #How to parce through intervals of the tuple in the array

['los', 'perros', 'azules', 'corrion']
[('los', 'da0mp0'), ('perros', 'ncmp000'), ('azules', 'aq0cp0'), ('corrion', None)]
[('los', 'da0mp0'), ('perros', 'ncmp000'), ('azules', 'aq0cp0'), ('corrion', None)]
[('los', 'da0mp0'), ('perros', 'ncmp000'), ('azules', 'aq0cp0'), ('corrion', None)]
('los', 'da0mp0')
los


In [ ]:
femSentence = "la niña es fea"
tokenized1 = femSentence.split()
print(tokenized1)
tagged3 = sp.pos_tag(tokenized1)
print(tagged3)

['la', 'niña', 'es', 'fea']
[('la', 'da0fs0'), ('niña', 'ncfs000'), ('es', 'vsip3s0'), ('fea', 'aq0fs0')]


In [ ]:
# Split corpus into training and testing set.
train = int(len(cess_sents)*90/100) # 90%

# Train a bigram tagger with only training data.
bi_tag = bt(cess_sents[:train], backoff=uni_tag)

# Evaluates on testing data remaining 10%
bi_tag.evaluate(cess_sents[train+1:])

# Using the tagger.
bi_tag.tag(Sentence.split(" "))

[('el', 'da0ms0'),
 ('perro', 'ncms000'),
 ('blanco', 'aq0ms0'),
 ('corrió', 'vmis3s0')]

In [ ]:
nltk.corpus.genesis.tagged_words()

AttributeError: ignored

# **Sanskrit Data**

Mounting Drive and importing text files of Sanskrit texts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gItA = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/bhagavadgItA.txt","r")
meghadhUta = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/meghadUta.txt","r")
rAmAyaNa = open("/content/drive/MyDrive/Colab Notebooks/ Final Project/Colab Notebooks/rAmAyaNa.txt", "r")

SyntaxError: ignored

comment to delete later: trying to figure out how to use repository here : https://github.com/ad2476/pos-research which has a sanskrit tagger. 

## PYCHARM CODE

In [ ]:
import os
import sys

import numpy
import IPython
from IPython import display

# making sure ghostscript works correctly so we can correctly convert .ps to .png for syntax tree
os.environ["PATH"] += "Library/usr/local/bin"
os.environ["PATH"] = "/usr/local/bin:" + os.environ["PATH"]
os.environ["PATH"] = "Library/usr/local/bin:" + os.environ["PATH"]


#import necessary toolboxes
import nltk
nltk.download('popular')
import pygame
from pygame.locals import *

#initialize user input of english sentence
userSentence = input("Please enter a sentence under 15 words:\n")

# create syntax tree that displays chunks
def syntaxTreeGenerator(array):
    for item in array:
        tokenized = nltk.word_tokenize(item)
        print("tokenized = ", tokenized)
        tagged = nltk.pos_tag(tokenized)
        print("tagged = ", tagged)

        chunkGram = r"""Chunk: {<DT\w?>*<JJ\w?>}""" #creating a chunk
        chunkParser = nltk.RegexpParser(chunkGram) #parsing the chunk

        chunked = chunkParser.parse(tagged)
        print(chunked)
        #chunked.draw()

        ### Save syntax tree as file ###
        from nltk.draw.tree import TreeView
        TreeView(chunked)._cframe.print_to_file('/Users/catherine/PycharmProjects/CLPS0950FinalProject/tree.ps')

        ### Convert .ps file to .png
        from PIL import Image
        psimage = Image.open(r'/Users/catherine/PycharmProjects/CLPS0950FinalProject/tree.ps')
        psimage.save(r'/Users/catherine/PycharmProjects/CLPS0950FinalProject/pygame_tree.png')




syntaxTreeGenerator([userSentence])

#initialize display window for pygame
pygame.init()
width, height = 1400, 720
screen = pygame.display.set_mode((width, height))

#define colors
black = (0,0,0)
white = (255, 255, 255)

# pygame window name
pygame.display.set_caption('Spanglish Generator')

#font
font = pygame.font.Font('freesansbold.ttf', 32)
font2 = pygame.font.Font('freesansbold.ttf', 20)

# text surface object
titleText = font.render('Spanglish Generator', True, black, white)
textRect = titleText.get_rect()

syntaxTreeText = font2.render('Syntax Tree', True, black, white)
rectSyntaxTreeText = syntaxTreeText.get_rect()
#rectSyntaxTreeText.w = width//2


# load images and center if need be
background = pygame.image.load("/Users/catherine/Desktop/project_background.png")
syntaxTree = pygame.image.load("/Users/catherine/PycharmProjects/CLPS0950FinalProject/pygame_tree.png")
rectSyntaxTree = syntaxTree.get_rect()
rectSyntaxTree.center = (width//2, 400)
screen_rect = screen.get_rect()

#rectSyntaxTree.center = (width//2, height//2)

# loop through
while True:
    #allows user to exit screen
    for event in pygame.event.get():
        # check if event is the X button
        if event.type == pygame.QUIT:
            # if yes then quite game
            pygame.quit()
            sys.exit()
    #clear the screen before putting things in again
    screen.fill(white)

    #draw screen elements
    for x in range(width // background.get_width() + int(1)):
        for y in range(height // background.get_height() + int(1)):
            screen.blit(background, (x * 100, y * 100))
    screen.blit(titleText, titleText.get_rect(midtop=screen_rect.midtop)) #says syntax generator at top of screen
    screen.blit(syntaxTreeText, (640, 260))
    #screen.blit(syntaxTreeText, syntaxTreeText.get_rect(w=rectSyntaxTreeText.w))
    screen.blit(syntaxTree, syntaxTree.get_rect(center=rectSyntaxTree.center))


    #screen.blit(print(syntaxTreeGenerator([userSentence])), (50, 50)) this doesn't work; shuts down


    #Update the screen
    pygame.display.flip()
    # loop through events
   # for event in pygame.event.get():
        #check if event is the X button
      #  if event.type==pygame.QUIT:
            # if yes then quite game
        #    pygame.quit()
          #  sys.exit()


            #quit()
        #pygame.display.update()






#initialize user input of english sentence
userSentence = input("Please enter a sentence:\n")
#userSentenceList = list(userSentence.split(" "))





# create syntax tree that displays chunks
def syntaxTreeGenerator(array):
    for item in array:
        tokenized = nltk.word_tokenize(item)
        print("tokenized = ", tokenized)
        tagged = nltk.pos_tag(tokenized)
        print("tagged = ", tagged)

        chunkGram = r"""Chunk: {<DT\w?>*<JJ\w?>}""" #creating a chunk
        chunkParser = nltk.RegexpParser(chunkGram) #parsing the chunk

        chunked = chunkParser.parse(tagged)
        print(chunked)
        chunked.draw()

#calling the function
exampleArray = ["The white dog ran"]
syntaxTreeGenerator([userSentence])
#syntaxTreeGenerator(exampleArray)




#for item in exampleArray:
  #  tokenized = nltk.word_tokenize(item)
  #  print(tokenized)
   # tagged = nltk.pos_tag(tokenized)
   # print(tagged)

   # chunkGram = r"""Chunk: {<DT\w?>*<JJ\w?>}""" #creating a chunk
   # chunkParser = nltk.RegexpParser(chunkGram) #parsing the chunk

   # chunked = chunkParser.parse(tagged)
   ## chunked.draw()

#for tree in chunkParser.parse(tagged): #creates a syntax tree for the parsed chunk
   # print(tree)
   # chunked.draw()